## Implementation of GoogleNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def inception_module(x, filters):
    # 1x1 convolution
    conv1x1 = layers.Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    # 3x3 convolution
    conv3x3 = layers.Conv2D(filters[1], (3, 3), padding='same', activation='relu')(x)

    # 5x5 convolution
    conv5x5 = layers.Conv2D(filters[2], (5, 5), padding='same', activation='relu')(x)

    # Max pooling
    maxpool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool_conv = layers.Conv2D(filters[3], (1, 1), padding='same', activation='relu')(maxpool)

    # Concatenate outputs
    output = layers.concatenate([conv1x1, conv3x3, conv5x5, maxpool_conv], axis=-1)

    return output

def googlenet(input_shape, num_classes):
    input_layer = layers.Input(shape=input_shape)

    # Initial convolutions
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception modules
    x = inception_module(x, [64, 128, 32, 32])
    x = inception_module(x, [128, 192, 96, 64])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = inception_module(x, [192, 208, 48, 64])
    x = inception_module(x, [160, 224, 64, 64])
    x = inception_module(x, [128, 256, 64, 64])
    x = inception_module(x, [112, 288, 64, 64])
    x = inception_module(x, [256, 320, 128, 128])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Flatten
    x = layers.Flatten()(x)

    # Fully connected layers
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=input_layer, outputs=x)

    return model

# Example usage:
input_shape = (224, 224, 3)  # Input shape of the images (height, width, channels)
num_classes = 1000  # Number of output classes
model = googlenet(input_shape, num_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)           0         ['max_pooling2d[0][0]']   